In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import dash_table
import numpy as np
from jupyter_dash import JupyterDash
import pdfkit
import base64
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pyodbc
from urllib.parse import quote_plus

In [2]:
# Parámetros de conexión
server = '10.0.0.133'
database = 'DB_GENERAL'
database2= 'DB_TRAMITE_DOCUMENTARIO'  # Reemplaza con el nombre de tu base de datos
username = 'sa'
password = 'Essalud23**'

def create_connection():
    conn = None
    try:
        # Establecer conexión
        conn = pyodbc.connect(r'DRIVER={ODBC Driver 18 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password};'
        'TrustServerCertificate=yes;')
        print("Conexión exitosa.")
    except Exception as e:
        print(f"Error al conectar: {e}")
    return conn

def create_connection2():
    conn = None
    try:
        # Establecer conexión
        conn = pyodbc.connect(r'DRIVER={ODBC Driver 18 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database2};'
        f'UID={username};'
        f'PWD={password};'
        'TrustServerCertificate=yes;')
        print("Conexión exitosa.")
    except Exception as e:
        print(f"Error al conectar: {e}")
    return conn

In [3]:
def fetch_data():
    conn = create_connection()
    if conn is None:
        raise ConnectionError("No se pudo establecer la conexión a la base de datos.")
    try:
        query = f"""
SELECT
    CASE 
        WHEN LTRIM(RTRIM(p.RAZON_SOCIAL)) <> '' AND p.RAZON_SOCIAL IS NOT NULL THEN p.RAZON_SOCIAL
        WHEN LTRIM(RTRIM(CONCAT(p.NOMBRES, ' ', p.APELLIDOS))) <> '' AND CONCAT(p.NOMBRES, ' ', p.APELLIDOS) IS NOT NULL THEN CONCAT(p.NOMBRES, ' ', p.APELLIDOS)
        ELSE 'No disponible' -- Puedes poner un valor predeterminado si todos los campos son vacíos
    END AS NOMBRE,
    p.NRO_DOCUMENTO,
    CLATEMP.DESCRIPCION AS CLASE_ORIGEN,
    td.DESCRIPCION AS TIPO_DOC,
    DTEMP.INDICATIVO_OFICIO AS DOC_REGISTRADO,
    DTEMP.NUM_TRAM_DOCUMENTARIO AS HOJA_TRAMITE,
    CLA.DESCRIPCION AS CLASE,
    ed.DESCRIPCION AS ESTADO,
    D.ASUNTO,
    FORMAT(MV.AUDIT_MOD, 'dd/MM/yyyy HH:mm:ss') AS FEC_ENVIO,
    O.DEPENDENCIA AS ORIGEN,
    FORMAT(MV.AUDIT_REC, 'dd/MM/yyyy HH:mm:ss') AS FEC_ACEPTACION,
    DE.DEPENDENCIA AS DESTINO,
    tt.descripcion AS ACCION
FROM DB_TRAMITE_DOCUMENTARIO.web_tramite.MOVIMIENTO_DOCUMENTO MV
    LEFT OUTER JOIN DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO DTEMP
        ON DTEMP.ID_DOCUMENTO = MV.ID_DOCUMENTO
    LEFT OUTER JOIN DB_GENERAL.dbo.PERSONA p 
        ON DTEMP.ID_PERSONA = p.ID
    LEFT OUTER JOIN DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO D
        ON D.ID_DOCUMENTO = MV.ID_OFICIO
    LEFT JOIN DB_TRAMITE_DOCUMENTARIO.dbo.CLASE_DOCUMENTO_INTERNO CLATEMP
        ON DTEMP.ID_CLASE_DOCUMENTO_INTERNO = CLATEMP.ID_CLASE_DOCUMENTO_INTERNO
    LEFT JOIN DB_TRAMITE_DOCUMENTARIO.dbo.CLASE_DOCUMENTO_INTERNO CLA 
        ON D.ID_CLASE_DOCUMENTO_INTERNO = CLA.ID_CLASE_DOCUMENTO_INTERNO
    LEFT OUTER JOIN DB_TRAMITE_DOCUMENTARIO.web_tramite.doc_tipo_tratamiento dtt 
        ON D.ID_DOCUMENTO = dtt.id_documento
    LEFT OUTER JOIN DB_TRAMITE_DOCUMENTARIO.web_tramite.TIPO_TRATAMIENTO tt 
        ON dtt.id_tipo_tratamiento = tt.id_tipo_tratamiento
    LEFT OUTER JOIN DB_GENERAL.JCARDENAS.H_DEPENDENCIA O 
        ON MV.ID_DEPENDENCIA_ORIGEN = O.CODIGO_DEPENDENCIA
    LEFT OUTER JOIN DB_GENERAL.JCARDENAS.H_DEPENDENCIA DE 
        ON MV.ID_DEPENDENCIA_DESTINO = DE.CODIGO_DEPENDENCIA
    LEFT OUTER JOIN DB_TRAMITE_DOCUMENTARIO.dbo.TIPO_DOCUMENTO td
        ON DTEMP.ID_TIPO_DOCUMENTO = td.ID_TIPO_DOCUMENTO
    LEFT OUTER JOIN DB_TRAMITE_DOCUMENTARIO.dbo.ESTADO_DOCUMENTO ed
        ON D.ID_ESTADO_DOCUMENTO = ed.ID_ESTADO_DOCUMENTO 
WHERE DTEMP.NUM_TRAM_DOCUMENTARIO = '00064528-2024'
AND p.NRO_DOCUMENTO = '76262790'
        """
        result = pd.read_sql(query, conn)
    finally:
        conn.close()  # Cerrar la conexión al final
    return result


In [5]:
df= fetch_data()

Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_10428\2230250958.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


In [6]:
df

,NOMBRE,NRO_DOCUMENTO,CLASE_ORIGEN,TIPO_DOC,DOC_REGISTRADO,HOJA_TRAMITE,CLASE,ESTADO,ASUNTO,FEC_ENVIO,ORIGEN,FEC_ACEPTACION,DESTINO,ACCION
0,ANTHONY BRANDON GAMARRA JAMANCA,76262790,CARTA,EXTERNO,006-ABGJ-2024,00064528-2024,None,None,None,01/08/2024 09:02:52,OFICINA DE GESTION DOCUMENTARIA,01/08/2024 15:17:55,GERENCIA DE SISTEMAS E INNOVACION TECNOLOGICA,None
1,ANTHONY BRANDON GAMARRA JAMANCA,76262790,CARTA,EXTERNO,006-ABGJ-2024,00064528-2024,INFORME,ACTIVO,GCTIC - GSIT - PAGO LOCACIÓN - 4504733316,05/08/2024 12:30:07,GERENCIA DE SISTEMAS E INNOVACION TECNOLOGICA,07/08/2024 12:47:23,GERENCIA DE SISTEMAS E INNOVACION TECNOLOGICA,None
2,ANTHONY BRANDON GAMARRA JAMANCA,76262790,CARTA,EXTERNO,006-ABGJ-2024,00064528-2024,NOTA,ACTIVO,GCTIC - GSIT - PAGO LOCACIÓN - 4504733316,09/08/2024 07:17:11,GERENCIA DE SISTEMAS E INNOVACION TECNOLOGICA,09/08/2024 11:18:21,GERENCIA CENTRAL DE TECNOLOGIAS DE INFORMACION...,CONTINUAR TRAMITE CORRESPONDIENTE
3,ANTHONY BRANDON GAMARRA JAMANCA,76262790,CARTA,EXTERNO,006-ABGJ-2024,00064528-2024,SIN DOCUMENTO,ACTIVO,GCTIC - GSIT - PAGO LOCACIÓN - 4504733316,09/08/2024 12:55:19,GERENCIA CENTRAL DE TECNOLOGIAS DE INFORMACION...,09/08/2024 12:55:19,GERENCIA CENTRAL DE TECNOLOGIAS DE INFORMACION...,None
4,ANTHONY BRANDON GAMARRA JAMANCA,76262790,CARTA,EXTERNO,006-ABGJ-2024,00064528-2024,PROVEIDO,ACTIVO,GCTIC - GSIT - PAGO LOCACIÓN - 4504733316,09/08/2024 14:44:50,GERENCIA CENTRAL DE TECNOLOGIAS DE INFORMACION...,09/08/2024 14:46:23,OFICINA DE ADMINISTRACION-GCTIC,PREPARAR RESPUESTA
5,ANTHONY BRANDON GAMARRA JAMANCA,76262790,CARTA,EXTERNO,006-ABGJ-2024,00064528-2024,PROYECTO DE RESPUESTA,ACTIVO,GCTIC - GSIT - PAGO LOCACIÓN - 4504733316,13/08/2024 08:30:55,OFICINA DE ADMINISTRACION-GCTIC,13/08/2024 08:30:55,OFICINA DE ADMINISTRACION-GCTIC,None
6,ANTHONY BRANDON GAMARRA JAMANCA,76262790,CARTA,EXTERNO,006-ABGJ-2024,00064528-2024,PROVEIDO,ACTIVO,GCTIC - GSIT - PAGO LOCACIÓN - 4504733316,13/08/2024 12:01:00,OFICINA DE ADMINISTRACION-GCTIC,13/08/2024 12:23:01,GERENCIA CENTRAL DE TECNOLOGIAS DE INFORMACION...,CONTINUAR TRAMITE CORRESPONDIENTE
7,ANTHONY BRANDON GAMARRA JAMANCA,76262790,CARTA,EXTERNO,006-ABGJ-2024,00064528-2024,MEMORANDO,ACTIVO,GCTIC - GSIT - PAGO LOCACIÓN - 4504733316,13/08/2024 16:16:41,GERENCIA CENTRAL DE TECNOLOGIAS DE INFORMACION...,13/08/2024 16:39:36,GERENCIA DE ABASTECIMIENTO,CONTINUAR TRAMITE CORRESPONDIENTE
8,ANTHONY BRANDON GAMARRA JAMANCA,76262790,CARTA,EXTERNO,006-ABGJ-2024,00064528-2024,PROVEIDO,ACTIVO,\t\nGCTIC - GSIT - PAGO LOCACIÓN - 4504733316,13/08/2024 16:59:52,GERENCIA DE ABASTECIMIENTO,13/08/2024 16:59:52,SUB GERENCIA DE ADQUISICIONES,CONTINUAR TRAMITE CORRESPONDIENTE
